# Tutorial 01: Dynamics and control of a Simple Pendulum

In this tutorial, you will implement forward and inverse dynamics of a simple pendulum system and study its applications in simulation as well as control.

<div style="display: flex; justify-content: space-around;">
    <div><img src="media/crane.jpg" width="400"></div>
    <div><img src="media/swing.gif" width="400"></div>
</div>

**Pre-requisites**

Knowledge of kinematic and dynamic models of simple robots, basic knowledge of integration schemes, underactuated robots.

**Goals**

Exploring the modelling and some control strategies for underactuated robots.

This notebook is organized as follows:

    1. Recap: Lagrangian Derivation of Equations of Motion (EOM)
    2. Derive the Solution to Forward and Inverse Dynamics
    3. Simulation using Forward Dynamics
    4. Gravity Compensation Control using Inverse Dynamics
    5. PID + Gravity Compensation Control
    6. Experiments on Pendulum Hardware 
      a. Free-fall experiment
      b. Gravity compensation 
      c. PID + Gravity Compensation 



## 1. Recap: Lagrangian Derivation of EOM

Consider a simple pendulum (undamped) as shown in the figure below. In the following, we will derive the EOM of the system using Python Symbolic Package called sympy.

<img src="media/pendulum_undamped_axes.png" width="400">

Physical parameters of the pendulum:
 * Mass: m
 * Length: l
 * Gravity: g

In [ ]:
import sympy 
import numpy as np

# Creates symbolic variable for physical parameters of the pendulum
m = sympy.Symbol('m')  # mass
l = sympy.Symbol('l')  # length
g = sympy.Symbol('g')  # gravity

The generalized coordinates and their derivatives are:
 * Angle: $q = \theta$
 * Angular Velocity: $\dot q = \dot{\theta}$
 * Angular Acceleration: $\ddot q = \ddot{\theta}$

In [ ]:
t = sympy.Symbol('t')  # Creates symbolic variable t for time

# Creates symbolic variables theta, thetadot and thetaddot
th = sympy.Function('theta')(t) 
th_dot = sympy.Function('theta_dot')(t) 
th_ddot = sympy.Function('theta_ddot')(t) 

 Recall that the Lagrangian of a mechanical system is defined as:
 $$
 L = K - V
 $$

In [ ]:
# Position Equation: r = [x, y]
r = np.array([0, 0]) # Type here
print("Position: ",r)

# Velocity Equation: d/dt(r) = [dx/dt, dy/dt]
v = np.array([r[0].diff(t), r[1].diff(t)])
print("Velocity: ",v)

# Develop the Lagrangian of the system
# Kinetic Energy
K = 0 # Type here
print("Kinetic Energy: ", sympy.simplify(K))

# Potential Energy 
V = 0 # Type here
print("Potential Energy: ", sympy.simplify(V))

# Lagrangian
L = K - V
print("Lagrangian: ", sympy.simplify(L))

 The Euler-Lagrange equations can be used to derive the Equations of Motion (EOM) of the system:
 $$
 \frac{d}{dt}\frac{\partial L}{\partial \dot{\mathbf{q}}} - \frac{\partial L}{\partial \mathbf{q}} = \tau
 $$
 The EOM of any mechanical system can be written in the canonical form:
 $$
 \mathbf{M} (\mathbf{q}) \ddot{\mathbf{q}} +  \mathbf{C} (\mathbf{q}, \dot{\mathbf{q}}) = \tau
 $$

In [ ]:
# Lagrange Terms:
dL_dth = L.diff(th)
dL_dth_dt = L.diff(th.diff(t)).diff(t)

# Euler-Lagrange Equations: dL/dq - d/dt(dL/ddq) = 0
eom = sympy.simplify(dL_dth_dt - dL_dth)

# Replace Time Derivatives and Functions with Symbolic Variables:
replacements = [(th.diff(t).diff(t), sympy.Symbol('th_ddot')), (th.diff(t),  sympy.Symbol('th_dot')), (th,  sympy.Symbol('th'))]
# Note: Replace in order of decreasing derivative order
eom = eom.subs(replacements)  # Use Subs method to substitute in the Symbolic Variables
print("tau = ", eom)

Additionally, we may add a damping term to account for the dissipation of energy in the joint of the pendulum. We can call this term $\tau_{d}$.

In this case, we model the damping as torque proportional and opposite in direction to $\dot{\mathbf{q}}$.

$$
\tau_{d} = -\mathbf{b} \dot{\mathbf{q}}
$$

$$
\mathbf{M} (\mathbf{q}) \ddot{\mathbf{q}} +  \mathbf{C} (\mathbf{q}, \dot{\mathbf{q}}) = \tau + \tau_{d}
$$

In [ ]:
b = sympy.Symbol('b')  # damping

eom = eom + b*th.diff(t)
eom = eom.subs(replacements)
print("tau = ", eom)

## 2. Forward and Inverse Dynamics

**Inverse Dynamics** is defined as the problem of solving for the generalized forces given the position, velocity and acceleration of the mechanical system in generalized coordinates. 
$$
\tau = \text{idyn}(\mathbf{q}, \dot{\mathbf{q}}, \ddot{\mathbf{q}}) = \mathbf{M} (\mathbf{q}) \ddot{\mathbf{q}} +  \mathbf{C} (\mathbf{q}, \dot{\mathbf{q}}) 
$$

In [ ]:
def inverse_dynamics(self, th, th_dot, th_ddot):
    """
    return torque acting on the revolute joint (tau) in terms of inputs
    use self.m, self.g, self.l, and self.b if needed
    """
    m = self.m
    g = self.g
    l = self.l
    b = self.b
    
    tau = 0.0 # Type Here!

    return tau

**Forward Dynamics** is defined as the problem of solving for the generalized acceleration of the mechanical system, given generalized position, velocity and forces acting on the system. 

$$
\ddot{\mathbf{q}} = \text{fdyn}(\mathbf{q}, \dot{\mathbf{q}}, \tau) = \mathbf{M}^{-1} (\mathbf{q}) \left( \tau -  \mathbf{C} (\mathbf{q}, \dot{\mathbf{q}}) \right)
$$

Note: Forward Dynamics of a mechanical system is a **second-order ordinary differential equation (ODE)**. 

In [ ]:
def forward_dynamics(self, th, th_dot, tau):
    """
    Return acceleration from current position, velocity and torque.
    """
    m = self.m
    g = self.g
    l = self.l
    b = self.b

    th_ddot = 0.0 # Type Here!
    
    return th_ddot

## 3. Simulation using Forward Dynamics

The state space of the system is the tuple of $\theta$ and its time derivative $\dot{\theta}$:
$$
\mathbf{x} = (\mathbf{q}, \dot{\mathbf{q}})
$$
The state space form of a dynamical system is given by:
$$
\dot{\mathbf{x}} = \mathbf{f}(\mathbf{x}, \mathbf{u})
$$
where $\mathbf{u}$ is the vector of control input. Note that the above form is a **first-order ODE**. 

Using the forward dynamics, one can deduce the first-order ODE of any mechanical system using the following technique:
$$
\dot{\mathbf{x}} = \mathbf{f}(\mathbf{x}, \mathbf{u}) = 
\begin{bmatrix}
\dot{\mathbf{q}}\\ 
\mathbf{M}^{-1} (\mathbf{q}) \left( \tau -  \mathbf{C} (\mathbf{q}, \dot{\mathbf{q}}) \right)
\end{bmatrix}
$$

The solution of this first-order ODE can be used to develop a dynamics simulator for a rigid multi-body system. While this can not be solved analytically, even for simpler systems, we can use numerical schemes to solve the ODEs.

Given an initial state $\mathbf{x}_0$, input  $\mathbf{u}$ and time step $dt$, we can predict the behavior of the mechanical system in the next time step using a numerical integration scheme. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math 

%matplotlib inline
from IPython.display import HTML

from pendulum_plant import PendulumPlant, plot_timeseries

The cell below constructs a pendulum object with the specified dynamic parameters. For now, our damping will be zero.

In [ ]:
mass = 0.06      # mass at the end of the pendulum [kg]
length = 0.1    # length of the rod [m]
damping = 0.0   # viscious friction [kg m/s]
gravity = 9.81  # gravity [kg m/s²]
inertia = mass*length*length   # inertia of the pendulum [kg m²]
torque_limit = np.inf  # torque limit of the motor, here we assume the motor can produce any torque

pendulum = PendulumPlant(mass=mass,
                         length=length,
                         damping=damping,
                         gravity=gravity,
                         inertia=inertia,
                         torque_limit=torque_limit)

# Add the forward dynamics method you implemented to the pendulum plant class 
PendulumPlant.forward_dynamics = forward_dynamics

# Add the inverse dynamics method you implemented to the pendulum plant class 
PendulumPlant.inverse_dynamics = inverse_dynamics

The first step we must take to design our integration scheme is to get our ODE.

In [ ]:
def f(self, t, x, tau):

    """
    Computes the rhs of the first order ODE.
    Hint: use the self.forward_dynamics function to compute acceleration
    """
    accn = 0.0 # Type Here!
    xdot = np.array([x[1], accn])
    return xdot # Return xdot

PendulumPlant.f = f

### 3.1 Explicit Euler integration scheme

Using Euler integration scheme leads to the following update rule: 
$$
\mathbf{x}[n+1] = \mathbf{x}[n] + \mathbf{f}(\mathbf{x}[n], \mathbf{u}[n]) dt
$$

Complete the following function implementing an explicit Euler integration scheme:

In [ ]:
def euler_integrator(self, t, x, dt, tau):

    x_new = np.zeros(2) # Type here
    return x_new

PendulumPlant.euler_integrator = euler_integrator

In order to simulate and animate a motion of the pendulum you can call the simulate_and_animate class method. The arguments are:

    - t0: the initial time
    - x0: the initial state vector containing position and velocity of the pendulum [th, th_dot]
    - tf: the final time
    - dt: the integration timestep
    - controller: controller object providing torques for the motor (introduced later)
    - integrator: the integration method. options are "euler" and "runge_kutta"
    
The method returns the trajectory split in to time (T), state (X) and torques (U) as well as an animation object.

Note: If you do not need the animation you can also call the method "simulate" with the same parameters. The method returns T, X, U but no animation object.

In [ ]:
%%capture
Tsim_euler, Xsim_euler, Usim_euler, anim_euler = pendulum.simulate_and_animate(
              t0=0.0,
              x0=[0.78, 0.0],
              tf=5.0,
              dt=0.002,
              controller=None,
              integrator="euler")

to show the animation in this notebook call (may take around a minute)

In [ ]:
HTML(anim_euler.to_html5_video())

You can use the plot_timeseries function to plot the trajectories (position, velocity and torque). Of course you can also plot these in any other way you like.

In [ ]:
plot_timeseries(Tsim_euler, Xsim_euler, Usim_euler)

As you can observe, the video rendering in Jupyter is quite slow. But simulating the pendulum is not. Hence, use the simulate() method instead which suppresses the animation but still solves the ODE for you.  

In [ ]:
Tsim_euler_s, Xsim_euler_s, Usim_euler_s = pendulum.simulate(
              t0=0.0,
              x0=[0.78, 0.0],
              tf=10.0,
              dt=0.002,
              controller=None,
              integrator="euler")
plot_timeseries(Tsim_euler_s, Xsim_euler_s, Usim_euler_s)

### 3.2 Fourth Order Runge-Kutta integration scheme

Using the fourth order Runge-Kutta integration scheme leads to the following update rule: 
$$
\mathbf{x}[n+1] = \mathbf{x}[n] + \frac{dt}{6}(\mathbf{k}_1 + 2\mathbf{k}_2 + 2\mathbf{k}_3 + \mathbf{k}_4)
$$

$$
t_{n+1} = t_n + dt
$$

where

$$
\mathbf{k}_1  =\mathbf{f}(t_{n}, \mathbf{x}_{n})
$$

$$
\mathbf{k}_2  =\mathbf{f}(t_{n} + \frac{dt}{2}, \mathbf{x}_{n} + dt \frac{\mathbf{k}_1}{2})
$$

$$
\mathbf{k}_3  =\mathbf{f}(t_{n} + \frac{dt}{2}, \mathbf{x}_{n} + dt \frac{\mathbf{k}_2}{2})
$$

$$
\mathbf{k}_4  =\mathbf{f}(t_{n} + dt, \mathbf{x}_{n} + dt \mathbf{k}_3)
$$

Complete the following function implementing an fourth order Runge-Kutta integration scheme:

In [ ]:
def runge_integrator(self, t, x, dt, tau):
    
    k1 = np.zeros(2) # Type Here
    k2 = np.zeros(2) # Type Here
    k3 = np.zeros(2) # Type Here
    k4 = np.zeros(2) # Type Here
    integ = np.zeros(2) # Type Here

    x_new = x + dt*integ

    return x_new
    
PendulumPlant.runge_integrator = runge_integrator

In [ ]:
%%capture
Tsim_RK4, Xsim_RK4, Usim_RK4, anim_RK4 = pendulum.simulate_and_animate(
              t0=0.0,
              x0=[0.78, 0.0],
              tf=5.0,
              dt=0.002,
              controller=None,
              integrator="runge_kutta")

In [ ]:
HTML(anim_RK4.to_html5_video())

In [ ]:
Tsim_RK4, Xsim_RK4, Usim_RK4 = pendulum.simulate(
              t0=0.0,
              x0=[1.57, 0.0],
              tf=10.0,
              dt=0.01,
              controller=None,
              integrator="runge_kutta")
plot_timeseries(Tsim_RK4, Xsim_RK4, Usim_RK4)

## Think-Pair-Share 

Try to play around with different initial conditions (x0), time step (dt), total simulation time (tf) choices! What do you observe?

1. Which integrator do you prefer?
2. What are the trade-offs of smaller vs bigger time steps?
3. What are the most interesting initial conditions? Can you already identify fixed points of the systems? What are attractors and repellors? Try adding damping to the simulation by modifying b in the definition of pendulum.
4. How accurate do you find these simulations? 

## 4. Gravity Compensation Control using Inverse Dynamics

### 4.1 Gravity Compensation

The gravity compensating controller cancels out the gravitational effect acting on the mass of the pendulum. With gravity compensation active, the pendulum will behave as it would in space.

The cell below shows the very basic structure of a controller class that the simulation method of the pendulum plant expects. Especially interesting is the get_control_output method. This method is called at every simulation step. Here the controller receives the current state of the pendulum and can compute a torque from that state, resulting in a reactive controller.

Please fill in the get_control method below, so that the returned torque cancels out the gravity effects acting on the pendulum mass.

In [ ]:
class GravityCompController():
    def __init__(self, mass, length, gravity):
        self.g = gravity        # gravitational acceleration on earth
        self.m = mass           # mass at the end of the pendulum
        self.l = length         # length of the rod

    def get_control_output(self, x):
        # compensate gravity with input torque
        m = self.m
        g = self.g
        l = self.l
        # Angle theta is available to you as a part of the state vector
        theta = x[0]
        
        ## Your Code Here        
        tau = 0.0  # Type Here! In terms of local variables m, g and l
        ##
        return tau

Next, we can initialize a controller object

In [ ]:
grav_con = GravityCompController(mass=mass,
                                 length=length,
                                 gravity=gravity)

and start a simulation with this controller

In [ ]:
Tsim_RK4, Xsim_RK4, Usim_RK4 = pendulum.simulate(
              t0=0.0,
              x0=[np.pi/2, 0.0],
              tf=10.0,
              dt=0.01,
              controller=grav_con,
              integrator="runge_kutta")
plot_timeseries(Tsim_RK4, Xsim_RK4, Usim_RK4)

If you implemented the torque response correctly, the pendulum should hold still at half height.

To see that the controller is actually doing something you can initialize the controller e.g. with 0.95*gravity and see the pendulum slowly falling down.

### 4.2 PID + Gravity compensation

Let's say we want to use gravity compensation to achieve a smooth swing-up motion with our pendulum. 

In [ ]:
class GravityPIDController():
    def __init__(self, mass, length, gravity, torque_limit, Kp, Ki, Kd, dt, grav_con):
        self.g = gravity        # gravitational acceleration on earth
        self.m = mass           # mass at the end of the pendulum
        self.l = length         # length of the rod
        
        self.Kp = Kp
        self.Ki = Ki
        self.Kd = Kd
        self.torque_limit = torque_limit
        
        self.dt = dt

        self.GC = grav_con # Gravity compensation controller
        
        self.errors = []

    def set_goal(self, goal=[np.pi, 0.0]):
        self.goal = goal

    def get_control_output(self, x):
        # Extract current state variables
        theta = x[0]  # current angle of the pendulum

        # Compute the error
        error = self.goal[0] - theta

        # Store the current error for the next iteration
        self.errors.append(error)

        # Compute the integral of the error
        integral = np.sum(self.errors) * self.dt

        # Compute the derivative of the error
        if len(self.errors) > 1:
            derivative = (self.errors[-1] - self.errors[-2]) / self.dt
        else:
            derivative = 0.0

        # Compute the control output
        tau = self.Kp * error + self.Ki * integral + self.Kd * derivative # PID version
        #tau = self.Kp * error - self.Kd * x[1] # PD version
        
        # add gravity compensation
        #tau += self.m * self.g * self.l * np.sin(theta)
        tau += self.GC.get_control_output(x)
        tau = np.clip(tau,-self.torque_limit, self.torque_limit)
        
        return tau

In [ ]:
# Gravity PID controller instance
grav_pid_con = GravityPIDController(mass, length, gravity, torque_limit = 0.14,
                                    Kp=0.01, Ki=0.001, Kd=0.001, dt=0.01, grav_con = grav_con)
grav_pid_con.set_goal([np.pi, 0.0])

In [ ]:
%%capture
Tsim_pid_gc, Xsim_pid_gc, Usim_pid_gc, anim_pid_gc = pendulum.simulate_and_animate(
              t0=0.0,
              x0=[0.001, 0.0],
              tf=10.0,
              dt=0.01,
              controller=grav_pid_con,
              integrator="runge_kutta")

In [ ]:
HTML(anim_pid_gc.to_html5_video())

In [ ]:
plot_timeseries(Tsim_pid_gc, Xsim_pid_gc, Usim_pid_gc)

In [ ]:
Tsim_pid_gc_s, Xsim_pid_gc_s, Usim_pid_gc_s = pendulum.simulate(
              t0=0.0,
              x0=[0.001, 0.0],
              tf=10.0,
              dt=0.01,
              controller=grav_pid_con,
              integrator="runge_kutta")
plot_timeseries(Tsim_pid_gc_s, Xsim_pid_gc_s, Usim_pid_gc_s)

## Think-Pair-Share

What happens if you modify the instance of the gravity PID controller? 

- Try adding a torque limit of 0.03 Nm, for instance. What do you see? Do you reach the desired position?
- Play around with the gains of the PID. What is the lowest torque limit for which the pendulum reaches the desired position?

## 5. Experiments on Pendulum Hardware

The following image shows a hardware prototype of an active pendulum system actuated with a small direct drive actuator (GL35 from T-motors with rated torque of 0.15 Nm and peak torque of 0.45 Nm). The control electronics are provided by MAB robotics. This system has been integrated in Chalmers Cloud infrastructure and students can remotely access the pendulum and get live experimental data from it with the comfort of their web-browsers. The maximum length of the experiment is **30 seconds** in each booking request. 

<img src="media/pendulum_hardware.jpeg" width="400">


In [ ]:
user_token = "Your User Token Here"

## 5a. Free-fall Experiment

In [ ]:
tf = 10 # Final time (s)
dt = 0.01 # Time step (s)
x0 = [1.57, 0.0] # Desired initial state. Note: desired velocity won't be respected in present implementation.
Treal_ff, Xreal_ff, Ureal_ff, Ureal_gc_des, video_path = pendulum.run_on_hardware(tf, dt, controller=None, user_token=user_token, x0 = x0) 

# Measured Position
plt.figure
plt.plot(Treal_ff, np.asarray(Xreal_ff).T[0])
plt.xlabel("Time (s)")
plt.ylabel("Position (rad)")
plt.title("Position (rad) vs Time (s)")
plt.show()

# Measured Velocity
plt.figure
plt.plot(Treal_ff, np.asarray(Xreal_ff).T[1])
plt.xlabel("Time (s)")
plt.ylabel("Velocity (rad/s)")
plt.title("Velocity (rad/s) vs Time (s)")
plt.show()

# Measured Torque
plt.figure
plt.plot(Treal_ff, Ureal_ff)
plt.xlabel("Time (s)")
plt.ylabel("Torque (Nm)")
plt.title("Torque (Nm) vs Time (s)")
plt.show()

# Export the data to csv file
measured_csv_data = np.array([Treal_ff, np.asarray(Xreal_ff).T[0], np.asarray(Xreal_ff).T[1],Ureal_ff]).T
np.savetxt("ff_measured_data.csv", measured_csv_data, delimiter=',', header="time,pos,vel,tau", comments="")

# Show the video
from IPython.display import Video
Video(video_path)

**Question: How does the behavior of the pendulum hardware differ from your simulation? What is missing in our previous plant model?**

## 6b. Gravity Compensation Experiment

Now we will test the gravity compensation controller that we developed directly on the real hardware. If your controller implementation is correct, you should be able to bring the pendulum by hand to any position in a quasi-static manner and it should hold itself. 

In [ ]:
tf = 10 # Final time (s)
dt = 0.01 # Time step (s)
x0 = [1.57, 0.0] # Desired initial state. Note: desired velocity won't be respected in present implementation.
# runs the controller on hardware
Treal_gc, Xreal_gc, Ureal_gc, Ureal_gc_des, video_path = pendulum.run_on_hardware(tf, dt, controller=grav_con, user_token=user_token, x0=x0) 

# Measured Position
plt.figure
plt.plot(Treal_gc, np.asarray(Xreal_gc).T[0])
plt.xlabel("Time (s)")
plt.ylabel("Position (rad)")
plt.title("Position (rad) vs Time (s)")
plt.show()

# Measured Velocity
plt.figure
plt.plot(Treal_gc, np.asarray(Xreal_gc).T[1])
plt.xlabel("Time (s)")
plt.ylabel("Velocity (rad/s)")
plt.title("Velocity (rad/s) vs Time (s)")
plt.show()

# Measured Torque
plt.figure
plt.plot(Treal_gc, Ureal_gc)
plt.plot(Treal_gc, Ureal_gc_des)
plt.xlabel("Time (s)")
plt.ylabel("Torque (Nm)")
plt.legend(['Measured Torque', 'Desired Torque'])
plt.show()

# Export the data to csv file
measured_csv_data = np.array([Treal_gc, np.asarray(Xreal_gc).T[0], np.asarray(Xreal_gc).T[1],Ureal_gc]).T
np.savetxt("gc_ff_measured_data.csv", measured_csv_data, delimiter=',', header="time,pos,vel,tau", comments="")

# Show the video
from IPython.display import Video
Video(video_path)

**Question: Even though the pendulum hardware and simulation behavior is different, why does the gravity compensation controller still work?**

## 6c. PID+Gravity Compensation Experiment

In [ ]:
tf = 10 # Final time (s)
dt = 0.01 # Time step (s)
# runs the controller on hardware
Treal_grav_pid_con, Xreal_grav_pid_con, Ureal_grav_pid_con, Ureal_grav_pid_con_des, video_path = pendulum.run_on_hardware(tf, dt, controller=grav_pid_con, user_token=user_token) 

Xdes_grav_pid_con = np.ones(int(tf/dt))*np.pi

# Measured Position
plt.figure
plt.plot(Treal_grav_pid_con, np.asarray(Xreal_grav_pid_con).T[0])
plt.plot(Treal_grav_pid_con, Xdes_grav_pid_con)
plt.xlabel("Time (s)")
plt.ylabel("Position (rad)")
plt.title("Position (rad) vs Time (s)")
plt.show()

# Measured Torque
plt.figure
plt.plot(Treal_grav_pid_con, Ureal_grav_pid_con)
plt.plot(Treal_grav_pid_con, Ureal_grav_pid_con_des)
plt.xlabel("Time (s)")
plt.ylabel("Torque (Nm)")
plt.legend(['Measured Torque', 'Desired Torque'])
plt.show()

# Show the video
from IPython.display import Video
Video(video_path)

**Question: When you put a torque limit of 0.03 Nm, why does not the swing up work? Can you make it work somehow?**